In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from math import sqrt

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
%matplotlib inline

In [108]:
train_df = pd.read_csv("train.csv", low_memory=False)
test_df = pd.read_csv("test.csv", low_memory=False)
data = pd.read_csv("data_set.csv", low_memory = False)

In [109]:
train_df = train_df[['id','precio']]
train_df['precio'].value_counts()

1500000.0    3041
2500000.0    3036
1200000.0    2909
3500000.0    2726
1300000.0    2480
             ... 
2704600.0       1
1352250.0       1
938260.0        1
2704460.0       1
2327000.0       1
Name: precio, Length: 15520, dtype: int64

In [110]:
data.head()

,id,habitaciones
0,254099,2.0
1,53461,3.0
2,247984,3.0
3,209067,2.0
4,185997,2.0


In [111]:
features = pd.merge(train_df, data, on='id', how='inner')
features = features.fillna(0)
features['precio'].value_counts()

1500000.0    3041
2500000.0    3036
1200000.0    2909
3500000.0    2726
1300000.0    2480
             ... 
2704600.0       1
1352250.0       1
938260.0        1
2704460.0       1
2327000.0       1
Name: precio, Length: 15520, dtype: int64

In [54]:
labels = features['precio']
features = features.drop(['id','precio'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.25)
print("Train: ",len(x_train),"Test: ",len(x_test))

Train:  180000 Test:  60000


In [55]:
neighbors = list(range(1,30))
train_results = []
test_results = []
best = [9999999999999,1]
mas_val = []
for n in neighbors:
    model = KNeighborsRegressor(n_neighbors = n)

    model.fit(x_train, y_train)
    pred=model.predict(x_test)
    error = sqrt(mean_absolute_error(y_test,pred))
    mas_val.append(error)
    print('MAS value for k= ' , n , 'is:', error)
    if best[0] > error:
            best[0] = error
            best[1] = n


MAS value for k=  1 is: 1328.8966756925336
MAS value for k=  2 is: 1305.1708761946334
MAS value for k=  3 is: 1338.967902727072
MAS value for k=  4 is: 1280.1420522478486
MAS value for k=  5 is: 1250.4433738931698
MAS value for k=  6 is: 1252.865689090415
MAS value for k=  7 is: 1241.7537916400258
MAS value for k=  8 is: 1235.7287370346833
MAS value for k=  9 is: 1280.731275527702
MAS value for k=  10 is: 1264.5724160178938
MAS value for k=  11 is: 1259.3894936610911
MAS value for k=  12 is: 1246.2664857880964
MAS value for k=  13 is: 1240.285149106281
MAS value for k=  14 is: 1241.6864675661504
MAS value for k=  15 is: 1240.4103167182668
MAS value for k=  16 is: 1232.1440799201575
MAS value for k=  17 is: 1242.202945946687
MAS value for k=  18 is: 1247.6230965026464
MAS value for k=  19 is: 1258.4428881237711
MAS value for k=  20 is: 1247.9576394470153
MAS value for k=  21 is: 1246.5864744627595
MAS value for k=  22 is: 1252.9034969279933
MAS value for k=  23 is: 1250.34861980859
MAS 

In [56]:
min(mas_val)

1232.1440799201575

In [57]:
best

[0, 1]

In [136]:
set_test = pd.merge(data, test_df[['id']],on = "id", how = "inner")
ids = set_test["id"]
set_test = set_test.drop(columns=['id'])

In [137]:
set_test = set_test.fillna(0)

In [138]:
set_test.head()

,habitaciones
0,3.0
1,1.0
2,2.0
3,2.0
4,2.0


In [139]:
model = KNeighborsRegressor(n_neighbors=16)
model.fit(x_train, y_train)

preds_kaggel_knn = model.predict(set_test)

In [167]:
df_kaggel_knn = pd.DataFrame(preds_kaggel_knn)
df_kaggel_knn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 1 columns):
0    60000 non-null float64
dtypes: float64(1)
memory usage: 468.9 KB


In [168]:
df_kaggel_knn = pd.DataFrame(df_kaggel_knn)

In [169]:
df_kaggel_knn.head()

,0
0,2.904562e+06
1,1.798091e+06
2,1.312736e+06
3,1.312736e+06
4,1.312736e+06


In [170]:
# df_kaggel_knn = df_kaggel_knn.to_frame()# df_kagg 
df_kaggel_knn["id"] = ids
df_kaggel_knn.rename(columns = {0: 'target'},inplace = True) 
df_kaggel_knn = df_kaggel_knn[['id','target']]

In [171]:
df_kaggel_knn.head()

,id,target
0,4941,2.904562e+06
1,51775,1.798091e+06
2,115253,1.312736e+06
3,299321,1.312736e+06
4,173570,1.312736e+06


In [172]:
df_kaggel_knn.to_csv('ResultadosKNN.csv',index=False)